In [9]:
#1-2) NLU 파일 자동 생성 (intent당 30개 샘플 → 빠르게)

In [1]:
import os, yaml
from collections import defaultdict
from datasets import load_dataset

In [2]:

base = "02_NLP_Concepts/Intent_Classification/C_rasa"
os.makedirs(base, exist_ok=True)

In [3]:
# 데이터 로드
ds = load_dataset("PolyAI/banking77")

Using the latest cached version of the dataset since PolyAI/banking77 couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /Users/jessicahong/.cache/huggingface/datasets/PolyAI___banking77/default/1.1.0/17ffc2ed47c2ed928bee64127ff1dbc97204cb974c2f980becae7c864007aed9 (last modified on Thu Aug 21 22:46:11 2025).


In [4]:

# nlu.yml (각 인텐트 30개 샘플만 사용해 속도 절약)
intents = defaultdict(list)
for text, label in zip(ds["train"]["text"], ds["train"]["label"]):
    intents[f"intent_{label}"].append(text)

In [5]:
nlu = {"version":"3.1","nlu":[]}
for intent, exs in intents.items():
    nlu["nlu"].append({"intent": intent, "examples": "\n".join(f"- {e}" for e in exs[:30])})
with open(f"{base}/nlu.yml","w",encoding="utf-8") as f:
    yaml.safe_dump(nlu, f, sort_keys=False, allow_unicode=True)

In [6]:
# config.yml (DIET + fallback, epoch 25로 가볍게)
config = {
  "version":"3.1","language":"en",
  "pipeline":[
    {"name":"WhitespaceTokenizer"},
    {"name":"LexicalSyntacticFeaturizer"},
    {"name":"CountVectorsFeaturizer","OOV_token":"[OOV]","min_ngram":1,"max_ngram":2},
    {"name":"CountVectorsFeaturizer","analyzer":"char_wb","min_ngram":3,"max_ngram":5},
    {"name":"DIETClassifier","epochs":25,"learning_rate":0.001},
    {"name":"FallbackClassifier","threshold":0.6,"ambiguity_threshold":0.1}
  ]
}

In [7]:
with open(f"{base}/config.yml","w",encoding="utf-8") as f:
    yaml.safe_dump(config, f, sort_keys=False, allow_unicode=True)

In [8]:
# domain.yml (NLU 평가만 하므로 최소)
with open(f"{base}/domain.yml","w",encoding="utf-8") as f:
    yaml.safe_dump({"version":"3.1","intents":["nlu_fallback"]}, f, sort_keys=False, allow_unicode=True)

print("Wrote:", [f"{base}/nlu.yml", f"{base}/config.yml", f"{base}/domain.yml"])

Wrote: ['02_NLP_Concepts/Intent_Classification/C_rasa/nlu.yml', '02_NLP_Concepts/Intent_Classification/C_rasa/config.yml', '02_NLP_Concepts/Intent_Classification/C_rasa/domain.yml']
